In [1]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import cophenet, fcluster
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

PREPROCESSED_PATH = './../preprocessed/'
IMAGES_PATH = './../images/'

FOR_TEST_PREDICTION = 20

In [ ]:
# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

# Plot dendrogram
def plot_dendrogram(Z, n_clusters):
    plt.figure(figsize=(40, 16))
    plt.title('Cluster Dendogram')
    plt.xlabel('Timeseries')
    plt.ylabel('Distance')
    hac.dendrogram(
        Z,
        truncate_mode='lastp',
        p=n_clusters,
        show_leaf_counts=True,
        show_contracted=True,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )
    plt.show()
    
def plot_silhouette(timeSeries, silhouette_avg, n_clusters, cluster_labels, pic_name=None):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 18)

    ax1.set_xlim([-0.1, 1])

    ax1.set_ylim([0, len(timeSeries) + (n_clusters + 3) * 10])
    y_lower = 10
    
    sample_silhouette_values = metrics.silhouette_samples(timeSeries, cluster_labels)
    silhouette_values = []
    
    for i in range(0, cluster_labels.max() + 1):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]
        
        if ith_cluster_silhouette_values.shape[0] > 0:
            ith_cluster_silhouette_values.sort()
            silhouette_values.append(ith_cluster_silhouette_values)

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples
        else:
            silhouette_values.append(pd.Series([]))

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-1.0, -0.9,  -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    if pic_name:
        fig.savefig(IMAGES_PATH + pic_name + '.png')
    plt.suptitle(("Silhouette analysis for Time Series Clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    return pd.Series(silhouette_values)

In [ ]:
# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'timeseries.csv')

In [ ]:
# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)
timeSeries = timeSeries.iloc[:-FOR_TEST_PREDICTION]

In [5]:
# Normalization between 0-1
normalization = MinMaxScaler()

# Z-score normalization
standardization = StandardScaler()

# Normalizing time series
timeSeriesNormalized = pd.DataFrame(normalization.fit_transform(timeSeries), index=timeSeries.index, columns=timeSeries.columns)
timeSeriesStandardized = pd.DataFrame(standardization.fit_transform(timeSeries), index=timeSeries.index, columns=timeSeries.columns)

In [6]:
X_normalized = timeSeriesNormalized.T
X_standardized = timeSeriesStandardized.T

# Applying ward method for clustering
Z_normalized = hac.linkage(X_normalized, method='ward')
Z_standardized = hac.linkage(X_standardized, method='ward')

In [ ]:
TEST_SILHOUETTE_AVG = True
        
if TEST_SILHOUETTE_AVG:
    normalized_avg = pd.DataFrame(columns=['n_clusters', 'avg'])
    normalized_min = pd.DataFrame(columns=['n_clusters', 'avg'])
    normalized_max = pd.DataFrame(columns=['n_clusters', 'avg'])
    
    standardized_avg = pd.DataFrame(columns=['n_clusters', 'avg'])
    standardized_min = pd.DataFrame(columns=['n_clusters', 'avg'])
    standardized_max = pd.DataFrame(columns=['n_clusters', 'avg'])
    for i in range(2, 100):
        cluster_labels = fcluster(Z_normalized, i, criterion="maxclust")
        avg = metrics.silhouette_score(X_normalized, cluster_labels, metric='euclidean')
        samples = pd.DataFrame({'cluster_labels': cluster_labels, 'coefficiente': metrics.silhouette_samples(X_normalized, cluster_labels)}).groupby(['cluster_labels'])
        
#         from_dendrogram = plot_silhouette(X_normalized, avg, i, cluster_labels)

        
        normalized_avg = normalized_avg.append(pd.DataFrame([[i, avg]], columns=['n_clusters', 'avg']), ignore_index=True)
        normalized_min = normalized_min.append(pd.DataFrame([[i, samples.mean().min()]], columns=['n_clusters', 'avg']), ignore_index=True)
        normalized_max = normalized_max.append(pd.DataFrame([[i, samples.mean().max()]], columns=['n_clusters', 'avg']), ignore_index=True)
        
        cluster_labels = fcluster(Z_standardized, i, criterion="maxclust")
        avg = metrics.silhouette_score(X_standardized, cluster_labels, metric='euclidean')
        samples = pd.DataFrame({'cluster_labels': cluster_labels, 'coefficiente': metrics.silhouette_samples(X_standardized, cluster_labels)}).groupby(['cluster_labels'])
        
        standardized_avg = standardized_avg.append(pd.DataFrame([[i, avg]], columns=['n_clusters', 'avg']), ignore_index=True)
        standardized_min = standardized_min.append(pd.DataFrame([[i, samples.mean().min()]], columns=['n_clusters', 'avg']), ignore_index=True)
        standardized_max = standardized_max.append(pd.DataFrame([[i, samples.mean().max()]], columns=['n_clusters', 'avg']), ignore_index=True)
        
    plt.figure(figsize=(30,16,))
    plt.title('Avaliando quantidade de clusters com coeficiente de silhueta', fontsize=35)
    plt.xlabel('Quantidade de clusters', fontsize=30)
    plt.ylabel('Coeficiente de silhueta', fontsize=30)
    plt.plot(normalized_avg.set_index('n_clusters'), label='Coeficiente médio')
    plt.plot(normalized_min.set_index('n_clusters'), label='Coeficiente mínimo')
    plt.plot(normalized_max.set_index('n_clusters'), label='Coeficiente máximo')
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.legend(loc='best', prop={'size': 30})
    plt.savefig(IMAGES_PATH + 'avg_min_max_normalization.png')
    plt.show()
    
    plt.figure(figsize=(30,16,))
    plt.title('Avaliando quantidade de clusters com coeficiente de silhueta', fontsize=35)
    plt.xlabel('Quantidade de clusters', fontsize=30)
    plt.ylabel('Coeficiente de silhueta', fontsize=30)
    plt.plot(standardized_avg.set_index('n_clusters'), label='Coeficiente médio')
    plt.plot(standardized_min.set_index('n_clusters'), label='Coeficiente mínimo')
    plt.plot(standardized_max.set_index('n_clusters'), label='Coeficiente máximo')
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.legend(loc='best', prop={'size': 30})
    plt.savefig(IMAGES_PATH + 'avg_min_max_standardization.png')
    plt.show()
    
    plt.figure(figsize=(30,16,))
    plt.title('Avaliando quantidade de clusters com coeficiente de silhueta', fontsize=35)
    plt.xlabel('Quantidade de clusters', fontsize=30)
    plt.ylabel('Coeficiente de silhueta', fontsize=30)
    plt.plot(normalized_avg.set_index('n_clusters')[1:], label='Coeficiente médio com séries normalizadas')
    plt.plot(standardized_avg.set_index('n_clusters')[1:], label='Coeficiente médio com séries z-normalizadas')
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.legend(loc='best', prop={'size': 30})
    plt.savefig(IMAGES_PATH + 'avaliacao_coef_silhueta.png')
    plt.show()

Normalized:  2 0.24759562281867598
Normalized:  3 0.18559553252605657
Normalized:  4 0.0990930217342644
Normalized:  5 0.1122187226553964
Normalized:  6 0.10743503378849065
Normalized:  7 0.11013020803890006
Normalized:  8 0.10795684484505406
Normalized:  9 0.11490404584621292
Normalized:  10 0.11389880585714272
Normalized:  11 0.11246892863454214
Normalized:  12 0.10780656844560443
Normalized:  13 0.1097336136540592
Normalized:  14 0.11651662248401999
Normalized:  15 0.0713159361703258
Normalized:  16 0.07253649274482671
Normalized:  17 0.0755587919709479
Normalized:  18 0.07552951221807223
Normalized:  19 0.07682747039982014
Normalized:  20 0.07763008002442073
Normalized:  21 0.08022013140326474
Normalized:  22 0.0743465734263002


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:522: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Normalized:  23 0.07451766976108919
Normalized:  24 0.07654326517028637
Normalized:  25 0.07831812604310738
Normalized:  26 0.07762513633520701
Normalized:  27 0.07960932836593002
Normalized:  28 0.08009008916841286
Normalized:  29 0.07457801889917927
Normalized:  30 0.07545010919885078
Normalized:  31 0.07719461231021071
Normalized:  32 0.07662269214184139
Normalized:  33 0.07701681508015588
Normalized:  34 0.07819470521532056
Normalized:  35 0.07931862711392082
Normalized:  36 0.0801378087274541
Normalized:  37 0.08018581840803593
Normalized:  38 0.0795739154776869
Normalized:  39 0.07999118490097151
Normalized:  40 0.07937414183055136
Normalized:  41 0.07831690622353038
Normalized:  42 0.07803572066739227
Normalized:  43 0.07874197968185057
Normalized:  44 0.0782232672800176
Normalized:  45 0.0800422011703614
Normalized:  46 0.08062974673358053
Normalized:  47 0.08054860678181558
Normalized:  48 0.07991436101530068
Normalized:  49 0.0759695753583281
Normalized:  50 0.077353973011757

In [10]:
# X_normalized.head()
# import tslearn.clustering.silhouette_score
# cluster_labels = fcluster(Z_normalized, 45, criterion="maxclust")
# silhouette_score(X_normalized, cluster_labels, metric="dtw")

import sys 
print(sys.executable)
!{sys.executable} -m pip install tslearn

C:\ProgramData\Anaconda3\python.exe
  Using cached https://files.pythonhosted.org/packages/95/da/eccb7d5a21bc38d77e99101e4fd6c0044a9fea7f4e254ac1c6ec209db29c/tslearn-0.1.24.tar.gz
  Running setup.py bdist_wheel for tslearn: started
  Running setup.py bdist_wheel for tslearn: still running...
  Running setup.py bdist_wheel for tslearn: still running...
  Running setup.py bdist_wheel for tslearn: still running...
  Running setup.py bdist_wheel for tslearn: still running...
  Running setup.py bdist_wheel for tslearn: finished with status 'done'
  Stored in directory: C:\Users\gabri\AppData\Local\pip\Cache\wheels\23\a3\65\51f5e2defdf863b80c4cfdb8714882e08193d3eedf011f1351
Successfully built tslearn


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
from tslearn.metrics import cdist_dtw
resultado = cdist_dtw(X_standardized.values)

In [ ]:
len(resultado)